**File explorer** to find the file you want to analyse with **Statement Processor** to generate all data and visualisations

In [ ]:
import traceback
import ipywidgets as widgets

%run ProcessxAPISGStatement.ipynb # notebook to process an xAPI-SG statement
%run widgets/selectFileWidget.ipynb
%run widgets/simvaWidget.ipynb
global location_file, filename, progressbarFile, progressbarTraces, checkboxesPlayersSelected
progressbarFile = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
progressbarTraces = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
location_file=""
filename=""

def is_json_and_not_list(str):
  try:
    return not isinstance(json.loads(str), list)
  except Exception as e:
    return False

#
# Loads either JSON-statement-per-line or JSON-array-of-statements from a str
# Updates progress in progress by calling progress.value from 0.0 to 1.0 (=finished)
# err_output & info_output can be either
#    None (= no output), 
#    a list (= ouput strings get appended), or
#    an object o where with o.output: print() is valid
#
def load_from_string(str, progress, info_output, err_output, players_info, timeformats):
    def log(target, o_str):
        if type(target) is None:
            pass
        elif type(target) is list:
            target.append(o_str)
        else:
            with target.output:
                print(o_str)

    count=0
    try:
        start_time = datetime.now()
        if is_json_and_not_list(str.partition('\n')[0]):
            total=len(str.splitlines())
            log(info_output, f"... 1st line is valid JSON; interpreting as one-statement-per-line ({total} statement(s))")
            # 1st line is well-formed json, and not json list of statements; assume 1-statement-per-line
            for s in str.splitlines():
                progress.value=count/total
                count+=1
                process_xapisg_statement(json.loads(s), players_info, timeformats)
        else:
            log(info_output, "... interpreting as statement-array")
            # attempt to process all of it as a single JSON document
            statements = json.loads(str)
            total = len(statements)
            log(info_output, f"... parsed correctly as json array, total statements = {total}")
            for s in statements:
                progress.value=count/total
                count+=1
                process_xapisg_statement(s, players_info, timeformats)
    except Exception as e:
        log(err_output, 
            f"ERROR loading at line/statement {count}/{total}: {e}\n"
            f"Full error:\n~~~\n{''.join(traceback.format_exception(e))}\n~~~\n"
            f"File must contain EITHER 1 statement (in JSON) per line, or be a well-formed JSON of statements. Please select another file.")
        return e
    progress.value=1.0
    log(info_output, f"... processed {count}/{total} statement(s) in {datetime.now() - start_time}. Displaying visualizations ...")
    return players_info




if storage == 'simva' :
    global simvaWidget
    #*** THE MAIN NOTEBOOK MUST HAVE A a variable of type ipyauth.Auth ***
    simvaWidget = SimvaBrowser(auth=a, storage_url=simva_storage_url, ca_file=simva_ca_file, accept='.json')

    def on_file_load(change):
        global location_file, checkboxesPlayersSelected, filename
        players_info.clear()
        # file with xAPI-SG statements
        location_file=simvaWidget.current_path
        basename=location_file.rpartition(simvaWidget.delimiter)[2]
        info = basename.partition('.')
        filename = info[0]
        try:
            load_from_string(
                simvaWidget.get_file_content(location_file), 
                progressbarTraces, 
                simvaWidget,
                simvaWidget,
                players_info, 
                timeformats)    
            if not isinstance(players_info, Exception):
                display_checkboxes()
                displayvis(None)
        except Exception as e:
            with simvaWidget.output:
                print(f"ERROR preparing to load: {e}\n"
                      f"Full error:\n~~~\n{''.join(traceback.format_exception(e))}\n~~~\n")
    simvaWidget.buttonRun.on_click(on_file_load)
else:
    if local:
        global fileBrowser
        #*** !TO BE USED WHEN THE NOTEBOOK IS RUN LOCALLY ***
        fileBrowser = FileBrowser(accept='.json')

        def on_file_load(change):
            try:
                global location_file, checkboxesPlayersSelected, filename
                players_info.clear()
                # file with xAPI-SG statements
                location_file=fileBrowser.path
                basename = os.path.basename(location_file)
                info = os.path.splitext(basename)
                filename = info[0]   
                str = open(f"{info[0]}.json", 'r', encoding='UTF-8').read()
                load_from_string(
                    str, 
                    progressbarTraces, 
                    fileBrowser,
                    fileBrowser,
                    players_info, 
                    timeformats)    
                if not isinstance(players_info, Exception):
                    display_checkboxes()
                    displayvis(None)
            except Exception as e:
                with fileBrowser.output:
                    print(f"ERROR preparing to load: {e}\n"
                          f"Full error:\n~~~\n{''.join(traceback.format_exception(e))}\n~~~\n")
        fileBrowser.buttonRun.on_click(on_file_load)

    else:
        global upload_button
        #***  !TO BE USED WHEN THE NOTEBOOK IS HOSTED ON THE WEB ***
        upload_button=widgets.FileUpload(
            description='Upload xAPI data',
            accept='.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=True,  # True to accept multiple files upload, else False
            layout=Layout(width='25%')
        )
        global outTabs,ErrorOut, uploadButtonApp
        outTabs=widgets.Output()
        ErrorOut=widgets.Output()
        uploadButtonApp=VBox([upload_button, progressbarFile, progressbarTraces, ErrorOut, outTabs])

        #Observe the file load widget (online version)
        def on_file_upload(change):
            outTabs.clear_output()
            ErrorOut.clear_output()
            upload_button._counter=0
            progressbarFile.value=0.0
            progressbarTraces.value=0.0
            filename=""
            number_of_files = len(upload_button.value) # number of json files selected
            players_info.clear()
            info_messages= []
            error_messages=[]
            with ErrorOut:
                for file_index in range(0,number_of_files):
                    progressbarFile.value = file_index/number_of_files
                    location_file = upload_button.value[file_index].name #get the file name from the embedded metadata dict
                    file = '-'.join(location_file.split(".")[:-1])
                    str = upload_button.value[file_index].content.tobytes().decode("utf-8") #extract data byte string and convert to str
                    filename += file + "-"
                    load_from_string(
                        str, 
                        progressbarTraces, 
                        info_messages,
                        error_messages,
                        players_info, 
                        timeformats)
                    progressbarFile.value = 1.0
                    progressbarTraces.value = 1.0
                # print & clear info messages
                info_str = '\n'.join(info_messages)
                info_messages = []
                print(f"{info_str}")
                # complain if error messages
                if len(error_messages) > 0:
                    err_str = '\n'.join(error_messages)
                    error_messages = []
                    print(f"ERROR loading {location_file}:\n{err_str}")
            with outTabs:
                clear_output(wait=True)
                display_checkboxes()
            displayvis(None)
            upload_button._counter=0
        upload_button.observe(on_file_upload, names='value')

def displayvis(change):
    if storage == 'simva' :
        with simvaWidget.output:
            displayAllVisualisations()
        simvaWidget.output.clear_output(wait=True)
    else:
        if local:
            with fileBrowser.output:
                displayAllVisualisations()
            fileBrowser.output.clear_output(wait=True)
        else:
            if not(players_info == {}) :
                ErrorOut.clear_output()
                with outTabs:
                    displayAllVisualisations()
                outTabs.clear_output(wait=True)